# 一、整体结构

CNN和之前介绍的神经网络一样，可以像乐高积木一样通过组装层来构建。不过，CNN中新出现了卷积层（Convolution层）和池化层（Pooling层）。卷积层和池化层将在下一节详细介绍，这里我们先看一下如何组装层以构建CNN。

之前介绍的神经网络中，相邻层的所有神经元之间都有连接，这称为全连接（fully-connected）。全连接层的实现就是之前的Affine层。

按照神经网络的结构构造一个网络，其结构如下：

![image-20230412162453899](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230412162453899.png)

按照CNN的结构构造网络，其结构如下：

![image-20230412162527511](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230412162527511.png)

如图所示，CNN 中新增了 Convolution 层 和 Pooling 层。CNN 的层连接顺序是“Convolution - ReLU -（Pooling）”（Pooling层有时会被省略）。此外，靠近输出的层中使用了之前的“Affine - ReLU”组合。


# 二、卷积层

## 2.1 全连接层的问题

全连接层存在的问题：数据的形状被“忽视”了。

比如，输入数据是图像时，图像通常是高、长、通道方向上的3维形状。但是，**向全连接层输入时，需要将3维数据拉平为1维数据**。实际上，前面提到的使用了MNIST数据集的例子中，输入图像就是1通道、高28像素、长28像素的（1, 28, 28）形状，但却被排成1列，以784个数据的形式输入到最开始的Affine层。**图像是3维形状，这个形状中应该含有重要的空间信息**。比如，空间上邻近的像素为相似的值、RBG的各个通道之间分别有密切的关联性、相距较远的像素之间没有什么关联等，3维形状中可能隐藏有值得提取的本质模式。**但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息。**

而**卷积层可以保持形状不变**。当输入数据是图像时，卷积层会以3维数据的形式接收输入数据，并同样以3维数据的形式输出至下一层。

另外，CNN 中，有时将卷积层的输入输出数据称为**特征图（feature map）**。其中，卷积层的输入数据称为**输入特征图（input feature map）**，输出数据称为**输出特征图（output feature map）**。



## 2.2卷积运算

卷积层进行的处理就是卷积运算。卷积运算相当于图像处理中的“**滤波器运算**”。在介绍卷积运算时，我们来看一个具体的例子：

![image-20230412210525047](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230412210525047.png)

如上图所示，卷积运算对输入数据应用滤波器。其实滤波器就是一个用若干个权重参数组成的小方块，在上图中滤波器的尺寸是3*3，所以权重参数一共有9个。卷积运算就是把这个滤波器（小方块）移到输入数据上，然后对应位置的元素相乘并求和，求和结果作为输出矩阵的一个元素，该过程如下图所示：

![image-20230413095923102](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413095923102.png)

其实和Affine层做的运算是类似的，都是乘权重然后求和，只不过这里数据的形状做了点变化，原来是一维的，现在是二维的。

对应的偏置项如下：

![image-20230413100151920](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413100151920.png)

## 2.3填充

根据上述的例子可以发现，卷积运算之后，输出特征图的尺寸是会变小的。所以如果要保持输入和输出的尺寸不变或者控制输出尺寸变大一点，就需要用到填充。

在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding），是卷积运算中经常会用到的处理。比如，在图7-6的例子中，对大小为(4, 4)的输入数据应用了幅度为1的填充。**“幅度为1的填充”是指用幅度为1像素的0填充周围。**

![image-20230413103845755](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413103845755.png)

>**使用填充主要是为了调整输出的大小**。比如，对大小为(4, 4)的输入数据应用(3, 3)的滤波器时，输出大小变为(2, 2)，相当于输出大小比输入大小缩小了 2个元素。这在反复进行多次卷积运算的深度网络中会成为问题。为什么呢？因为**如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为1，导致无法再应用卷积运算**。为了避免出现这样的情况，就要使用填充。在刚才的例子中，将填充的幅度设为 1，那么相对于输入大小(4, 4)，输出大小也保持为原来的(4, 4)。因此，卷积运算就可以在保持空间大小不变的情况下将数据传给下一层。

## 2.4步幅

应用滤波器的位置间隔称为步幅（stride）：

![image-20230413155419080](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413155419080.png)

## 2.5关于尺寸的计算

在设计CNN网络结构的时候，往往需要预先知道输出特征图的尺寸，所以输入特征图尺寸、步幅、滤波器尺寸、输出特征图尺寸之间的关系就很重要，下面来推导一下他们之间的关系式：

![image-20230413162547835](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413162547835.png)

如图所示，$N$表示输入特征图的边长（这里边长将输入特征图的$height$和$width$统一写成了$N$，用的时候宽高不一定相等），$No$表示输出特征图的边长，$stride$表示步幅，$n$是滤波器边长。现在到移动停止为止需要移动$k$次，则可知$k$次后，滤波器第一列在输入特征图中所处位置为：

$$
1+stride*k
$$

所以，最后一列在输入特征图中所处位置为：

$$
1+stride*k+n-1=stride*k+n
$$

同时，第$k$次是最后一次移动的判断条件是：第$k$次移动后，滤波器的最后一列不会超出输入特征图的最后一列；且第$k+1$次移动后，最后一列超出最后一列：

$$
stride*k+n<=N \\
stride*(k+1)+n>N
$$

整理可得：

$$
\frac{N-n}{stride}-1<k<=\frac{N-n}{stride}
$$

得出$k$的取值为：

$$
k=\lfloor \frac{N-n}{stride} \rfloor
$$

最后，输入特征图的边长表示如下：

$$
No=\lfloor \frac{N-n}{stride} \rfloor+1
$$

【注】：上述公式没加填充的因素，如果要加，就在输入特征图边长$N$上多加一个$2*padding$（$padding$是填充量）。

【举例】：输入大小：(28, 31)；填充：2；步幅：3；滤波器大小：(5, 5)

$$
W=\frac{(28+2*2)-5}{3}+1=10 \\
H=\frac{(31+2*2)-5}{3}+1=11
$$

## 2.6三维数据的卷积运算

在讲三维数据之前，首先需要了解一下一张彩色图像是怎么构成的？ 众所周知，一张图像是由很多个像素点构成的。根据三原色原理，任何颜色都能用红、绿、蓝三原色调出来，所以彩色图像的每个像素点也都是要用这三种颜色调出来的，这三原色在计算机中称为R(red)、G(green)、B(blue)三通道。每个像素点都由三个个颜色组成，所以一张彩色图就由三个通道堆叠组成，如下图所示：

![image-20230413174253336](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413174253336.png)

下面回到卷积的内容上，通道方向上有多个特征图时，会按通道进行输入数据和滤波器的卷积运算，并将结果相加，从而得到输出。

![image-20230413174432909](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413174432909.png)

![image-20230413174510385](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230413174510385.png)

【注意】：在3维数据的卷积运算中，**输入数据和滤波器的通道数要设为相同的值**。同时，**每个通道的滤波器大小要全部相同**。

## 2.7三维数据的形状表示

把3维数据表示为多维数组时，书写顺序为（channel, height, width）。比如，通道数为C、高度为H、长度为W的数据的形状可以写成（C, H, W）。滤波器也一样，要按（channel, height, width）的顺序书写。比如，通道数为C、滤波器高度为FH（Filter Height）、长度为FW（Filter Width）时，可以写成（C, FH, FW）。

![image-20230414090817565](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414090817565.png)

在上图中，数据输出是1张特征图。所谓1张特征图，换句话说，就是通道数为1的特征图。那么，如果要在通道方向上也拥有多个卷积运算的输出，该怎么做呢？为此，就需要用到**多个滤波器**（权重）。

![image-20230414091118487](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414091118487.png)

如上图所示，关于卷积运算的滤波器，也必须考虑滤波器的数量。因此，作为4维数据，滤波器的权重数据要按 **(output_channel, input_channel, height, width)** 的顺序书写。比如，通道数为3、大小为5 × 5的滤波器有20个时，可以写成(20, 3, 5, 5)。

卷积运算中（和全连接层一样）存在偏置。在上图的例子中，如果进一步追加偏置的加法运算处理，则结果如下图所示：

![image-20230414091802851](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414091802851.png)

这里看到偏置项和第一次的输出的形状不同，怎么能实现相加？这里就运用到**Numpy的广播机制**：NumPy中，形状不同的数组之间也可以进行运算。

![image-20230414094207979](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414094207979.png)

总而言之，就是先扩展，然后实现元素级别运算。回到卷积运算中，偏置项$(FN,1,1)$会水平，竖直扩展成$(FN,OH,OW)$，简单来说就是**每一个通道，都要加上同一个值**。

## 2.8批处理

之前的全连接神经网络的实现对应了批处理，我们希望卷积运算也同样对应批处理。为此，需要将在**各层间传递的数据保存为4维数据（一条数据是3维，有N条所以是4维，多的一维是数据的条数）**。具体地讲，就是按 **(batch_num, channel, height, width)** 的顺序保存数据。

![image-20230414101144961](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414101144961.png)

# 三、池化层

池化是缩小高、长方向上的空间的运算。下图进行的是将2 × 2的区域集约成1个元素的处理，缩小空间大小，按步幅2进行2 × 2的Max池化时的处理顺序。

![image-20230414103648363](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414103648363.png)

一般来说，**池化的窗口大小会和步幅设定成相同的值**。比如，3 × 3的窗口的步幅会设为3，4 × 4的窗口的步幅会设为4等。

>除了Max池化之外，还有Average池化等。相对于Max池化是从目标区域中取出最大值，Average池化则是计算目标区域的平均值。在图像识别领域，主要使用Max池化。因此，本书中说到“池化层”时，指的是Max池化。

**池化层的特征**

- 没有要学习的参数

- 通道数不发生变化

    ![image-20230414104229565](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414104229565.png)

- 对微小的位置变化具有鲁棒性（健壮）

    输入数据发生微小偏差时，池化仍会返回相同的结果。因此，池化对输入数据的微小偏差具有鲁棒性。比如，3 × 3的池化的情况下，如下图所示，池化会吸收输入数据的偏差：

    ![image-20230414104347053](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414104347053.png)

    输入数据在宽度方向上只偏离1个元素时，输出仍为相同的结果。

# 四、卷积层与池化层的实现


## 4.1四维数组

首先来看一下四维数据的使用：

In [2]:
import numpy as np

x=np.random.rand(10,1,28,28) # 10个单通道28*28数据（10个灰度图）

print(x.shape) # 查看x的形状

print(x[0]) # 查看x中第一条数据

(10, 1, 28, 28)
[[[0.89477601 0.42078335 0.83068344 0.31242428 0.89812909 0.90619721
   0.54501538 0.25691006 0.8878828  0.64946369 0.64018247 0.41833143
   0.12655946 0.18512659 0.69615588 0.06668205 0.34518392 0.47677963
   0.7773527  0.97389499 0.40991714 0.37686073 0.27789164 0.94737628
   0.71636654 0.11933569 0.80524605 0.42224317]
  [0.20851567 0.96926236 0.78410033 0.5513441  0.08519034 0.91275512
   0.6066     0.61426685 0.7881379  0.82330811 0.66509579 0.90337829
   0.31987286 0.49873569 0.86187703 0.54181071 0.04532741 0.70551471
   0.09109872 0.81281964 0.9154662  0.02895876 0.2807245  0.68508463
   0.80744845 0.44591847 0.26894088 0.81431507]
  [0.61465449 0.28246968 0.04949361 0.52256625 0.74277186 0.19768757
   0.05975234 0.62252855 0.39429718 0.11465035 0.06841329 0.05937435
   0.36725144 0.25865115 0.82749424 0.76793746 0.89127583 0.69345718
   0.1371296  0.48625334 0.90851392 0.46619112 0.01788243 0.7622822
   0.52148608 0.84683201 0.80228181 0.02643945]
  [0.36592758

## 4.2 基于im2col的展开

如果老老实实地实现卷积运算，要重复好几层的for语句，这样的实现有点麻烦，而且，NumPy中存在使用for语句后处理变慢的缺点（**NumPy中，访问元素时最好不要用for语句**）。这里，我们不使用for语句，而是使用im2col这个便利的函数进行简单的实现。

im2col是一个函数，**将输入数据展开以适合滤波器**（权重）。如下图所示，对3维的输入数据应用im2col后，数据转换为2维矩阵（正确地讲，是把包含批数量的4维数据转换成了2维数据）。

![image-20230414135358651](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414135358651.png)

im2col会在所有应用滤波器的地方进行这个展开处理，

![image-20230414164642417](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414164642417.png)

使用im2col展开输入数据后，之后就只需将卷积层的滤波器（权重）纵向展开为1列，并计算2个矩阵的乘积即可。这和全连接层的Affine层进行的处理基本相同。

![image-20230414164734607](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414164734607.png)

卷积运算的滤波器处理的细节：将滤波器纵向展开为1列，并计算和im2col展开的数据的矩阵乘积，最后转换（reshape）为输出数据的大小

In [3]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : 由(数据量, 通道, 高, 长)的4维数组构成的输入数据
    filter_h : 滤波器的高
    filter_w : 滤波器的长
    stride : 步幅
    pad : 填充

    Returns
    -------
    col : 2维数组
    """

    # 先计算输出特征图的尺寸
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1 #'//'表示先做除法，再向下取整
    out_w = (W + 2*pad - filter_w)//stride + 1

    
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

x1 = np.random.rand(2, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=2, pad=0)
print(col1.shape)


(8, 75)


上述代码的解释：

```py
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1
```

这两行计算输出特征图的高度 out_h 和宽度 out_w。

```py
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
```

这一行使用 np.pad 函数在输入数据的四个维度上添加填充。它接收四个参数：`input_data`（要填充的数组）、`[(0,0), (0,0), (pad, pad), (pad, pad)]`（指定在每个轴上添加的填充数量）、`'constant'`（指定填充类型为常数）和 `0`（指定填充值为 0）。

`[(0,0), (0,0), (pad, pad), (pad, pad)]` 是 np.pad 函数的第二个参数 `pad_width` 的值。它指定了在每个轴上添加的填充数量。在这种情况下，输入数据是一个四维数组，因此 `pad_width` 参数也是一个四元组，每个元素对应一个轴。每个元组中的两个值分别指定了在该轴的开头和结尾添加的填充数量。

例如，在 im2col 函数中，pad_width 的值为 `[(0,0), (0,0), (pad, pad), (pad, pad)]`。这意味着在第一个轴和第二个轴上不添加填充（填充数量为 0），而在第三个轴和第四个轴上，在开头和结尾都添加 pad 个填充值。

`col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))`的作用是保存滤波器每个位置移动过程中，所经过的位置。现在对这个可能无法理解，在后面会详细讲解。

```py
for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
```

这个for循环是该函数的关键。`y_max = y + stride*out_h`的意思是，行初始位置`y`移动`out_h`次后，到达最终停止位置。实际上，应该是**到达的最终位置的后stride行**（类似**左闭右开**的意思，多移了一次）。因为我们在（2.5）节计算尺寸的时候，算出输出特征图的边长`out_h`是比移动次数`k`多1的。这里要多往后取一次的原因是，数组的切片索引都是左闭右开`y:y_max:stride`。

所以，

```py
y_max = y + stride*out_h
x_max = x + stride*out_w
```

这两句话用两个for循环共同确定了滤波器中每个位置的元素，在经过移动后，最终到达的位置。求最终到达的位置`x_max`,`y_max`的作用是什么呢？是为了使用切片索引`x:x_max:stride`,`y:y_max:stride`，从而求出滤波器中每个元素在移动过程中经过的位置。

例如：现有输入特征图$shape=(N=1,C=3,H=7,W=7)$，步幅$stride=2$，滤波器$shape=(C=3,H=5,W=5)$，填充$pad=0$：

![image-20230415195618867](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230415195618867.png)

通过计算得到输出特征图$(C=3,out\_h=2,out\_w=2)$，假设现在取第一行第二个元素$y=0$，$x=1$。所以，$y\_max=y+stride*out\_h=4$，$x\_max=x+stride*out\_w=5$：

![image-20230415200525482](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230415200525482.png)

所以，`img[:, :, y:y_max:stride, x:x_max:stride]`取到的元素如下：

![image-20230415210509263](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230415210509263.png)

同时，由于$InputImg$是一个四维矩阵$(1,3,7,7)$，即1张3通道的图像。所以，`img[:, :, y:y_max:stride, x:x_max:stride]`得到的$Img\_$的中，图像数量和通道数量都保持不变，即：新矩阵$shape=(1,3,2,2)$。

现在，获取到的这个矩阵$Img\_$要存在哪里呢？这里就用到之前讲的六维矩阵$col$。观察新矩阵$Img\_$的形状$shape=(1,3,2,2)$，很容易发现，1就是$N$的取值，3就是$C$的取值，2就是输出特征图$h$和$w$的长度，所以$Img\_$的形状$shape=(N,C,out\_h,out\_w)$。又因为保存的时候，$col$某几维的形状要和$Img\_$的形状相同，所以$col$要设计成$(N, C, filter\_h, filter\_w, out\_h, out\_w)$，其中$filter\_w, out\_h$表示是滤波器的哪一个位置的元素。

所以，保存$Img\_$应该写为：

```py
col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
```

其中，$y,x$指示的是滤波器中第几行第几列的元素，在上面的例子中，$y=0,x=1$。

总而言之，上面这么多步骤，就是把滤波器中每个元素在移动过程中的可能取值全部保存下来了。

```py
col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
```

$transpose(0, 4, 5, 1, 2, 3)$ 是一个在 NumPy数组上调用的方法，它重新排列数组的维度。括号内的数字表示轴的新顺序。在这种情况下，第一个轴（0）保持不变，第二个轴（1）移动到第四个位置，第三个轴（2）移动到第五个位置，依此类推。即：$(N, C, filter\_h, filter\_w, out\_h, out\_w)$变成$(N, out\_h, out\_w, C, filter\_h, filter\_w)$

reshape函数执行的过程如下$(x.shape=(2,3,7,7),stride=2,pad=0,fileter.shape=(3,5,5))$：

<img src="https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230416111416405.png" alt="image-20230416111416405" style="zoom:150%;" />

由上图可知，六维数组$col$就可以看成是**两个三维数组的嵌套**（一个三维数组的元素还是一个三维数组），所以**六维数组变二维就是做两次三维数组到一维的转换**（图中的2处3D to 1D）。同时，很容易就能发现，三维数组$(C,filter\_h,filter\_w)$就是输入特征图中**每一次应用滤波器的地方(即与滤波器进行计算的部分)**，因为有8个这样的三维数组，所以最后输出的纵轴长度是8，横轴方向就是把这个三维数组拉成一维数组。

![image-20230414164642417](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414164642417.png)

![image-20230414164734607](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230414164734607.png)

有了上面的解释，现在就能理解为什么在$reshape$之前要对$col$进行`transpose(0, 4, 5, 1, 2, 3)`了。因为一开始col数组是针对滤波器中每一个元素在移动过程中会应用的地方，而通过`transpose(0, 4, 5, 1, 2, 3)`转换之后，就整合成了针对一个完整滤波器在移动过程中会应用的地方。即：从局部到整体的变化。

下面用im2col实现卷积层：

In [6]:
import sys,os

current_file_path = os.path.dirname(os.path.realpath('__file__')) # 获取ipynb文件的路径
Demo_path = os.path.join(current_file_path, 'Demo') # 组合成Demo文件夹所在路径
sys.path.append(Demo_path) # 添加路径

from common.util import im2col

class Convolution:
    def __init__(self,W,b,stride=1,pad=0) -> None:
        self.W=W
        self.b=b
        self.stride=stride
        self.pad=pad

    def forward(self,x):
        FN,C,FH,FW=self.W.shape
        N,C,H,W=x.shape
        
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 滤波器的展开
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

## 4.3池化层的实现



In [28]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        # 展开(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        # 最大值(2)
        out = np.max(col, axis=1)
        # 转换(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        return out
    
# x=np.array([[[[1,2,3,0],
#               [0,1,2,4],
#               [1,0,4,2],
#               [3,2,0,1]],
#              [[3,0,6,5],
#               [4,2,4,3],
#               [3,0,1,0],
#               [2,3,3,1]],
#              [[4,2,1,2],
#               [0,1,0,4], 
#               [3,0,6,2],
#               [4,2,4,5]]]])
# pooling = Pooling(2,2,2,0)
# out=pooling.forward(x)

原图配的有误，所以整个计算过程我自己重新画了一张：

<img src="https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230417145858649.png" alt="image-20230417145858649" style="zoom:150%;" />

这里解释一下tranpose的实现原理，上图中是四维的数据，这里就省略了维度$N$，仅讨论三维的情况：

![image-20230417155148754](https://zyc-learning-1309954661.cos.ap-nanjing.myqcloud.com/machine-learning-pic/image-20230417155148754.png)

从上图中可以发现，$transpose$就是确定了每个维度（轴axis）的先后顺序。$transpose$前，`(C,H,W)=(axis0,axis1,axis2)`，现在想要将输出图像变成如图所示的样子，就要调整轴的位置，新图的$C'$就是原来的$W$，$H'$就是原来的$C$，$W'$就是原来的$H$。调整后，`(C',H',W')=(W,H,C)=(axis2,axis0,axis1)`，所以`transpose(2,0,1)`